In [2]:
# Importing the necessary modules from the chromadb package:
# chromadb is used to interact with the Chroma DB database,
# embedding_functions is used to define the embedding model
import chromadb
from chromadb.utils import embedding_functions

In [3]:
# Define the embedding function using SentenceTransformers
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

/Users/sanielbhattarai/Desktop/IBM Certification/Vector Databases for RAG/Module-1/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Create a new instance of ChromaClient to interact with the Chroma DB
client = chromadb.Client()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [5]:
# Define the name for the collection to be created or retrieved
collection_name = "my_grocery_collection"

In [6]:
# Function to perform a similarity search in the collection
def perform_similarity_search(collection, all_items):
    try:
        # Define the query term you want to search for in the collection
        query_term = "apple"
        

        # Perform a query to search for the most similar documents to the 'query_term'
        results = collection.query(
            query_texts=[query_term],
            n_results=3  # Retrieve top 3 results
        )
        print(f"Query results for '{query_term}':")
        print(results)
        
        # Check if no results are returned or if the results array is empty
        if not results or not results['ids'] or len(results['ids'][0]) == 0:
            # Log a message indicating that no similar documents were found for the query term
            print(f'No documents found similar to "{query_term}"')
            return
        # print(results)

        print(f'Top 3 similar documents to "{query_term}":')
        # Access the nested arrays in 'results["ids"]' and 'results["distances"]'
        for i in range(min(3, len(results['ids'][0]))):
            doc_id = results['ids'][0][i]  # Get ID from 'ids' array
            score = results['distances'][0][i]  # Get score from 'distances' array
            # Retrieve text data from the results
            text = results['documents'][0][i]
            if not text:
                print(f' - ID: {doc_id}, Text: "Text not available", Score: {score:.4f}')
            else:
                print(f' - ID: {doc_id}, Text: "{text}", Score: {score:.4f}')
        # pass 
    except Exception as error:
        print(f"Error in similarity search: {error}")

In [7]:
# Define the main function to interact with the Chroma DB
def main():
    try:
        # Create a collection in the Chroma database with a specified name, 
        # distance metric, and embedding function. In this case, we are using 
        # cosine distance
        collection = client.create_collection(
            name=collection_name,
            metadata={"description": "A collection for storing grocery data"},
            configuration={
                "hnsw": {"space": "cosine"},
                "embedding_function": ef
            }
        )
        print(f"Collection created: {collection.name}")

        # Array of grocery-related text items
        texts = [
            'fresh red apples',
            'organic bananas',
            'ripe mangoes',
            'whole wheat bread',
            'farm-fresh eggs',
            'natural yogurt',
            'frozen vegetables',
            'grass-fed beef',
            'free-range chicken',
            'fresh salmon fillet',
            'aromatic coffee beans',
            'pure honey',
            'golden apple',
            'red fruit'
        ]

        # Create a list of unique IDs for each text item in the 'texts' array
        # Each ID follows the format 'food_<index>', where <index> starts from 1
        ids = [f"food_{index + 1}" for index, _ in enumerate(texts)]

        # Add documents and their corresponding IDs to the collection
        # The `add` method inserts the data into the collection
        # The documents are the actual text items, and the IDs are unique identifiers
        # ChromaDB will automatically generate embeddings using the configured embedding function
        collection.add(
            documents=texts,
            metadatas=[{"source": "grocery_store", "category": "food"} for _ in texts],
            ids=ids
        )
        
        # Retrieve all the items (documents) stored in the collection
        # The `get` method fetches all data from the collection
        all_items = collection.get()
        # Log the retrieved items to the console for inspection
        # This will print out all the documents, IDs, and metadata stored in the collection
        print("Collection contents:")
        print(f"Number of documents: {len(all_items['documents'])}")
        
        perform_similarity_search(collection, all_items)
        # pass

    except Exception as error:  # Catch any errors and log them to the console
        print(f"Error: {error}")

if __name__ == "__main__":
    main()

Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Collection created: my_grocery_collection
Collection contents:
Number of documents: 14
Query results for 'apple':
{'ids': [['food_13', 'food_1', 'food_14']], 'embeddings': None, 'documents': [['golden apple', 'fresh red apples', 'red fruit']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'source': 'grocery_store', 'category': 'food'}, {'category': 'food', 'source': 'grocery_store'}, {'source': 'grocery_store', 'category': 'food'}]], 'distances': [[0.38246476650238037, 0.4808924198150635, 0.5965152382850647]]}
Top 3 similar documents to "apple":
 - ID: food_13, Text: "golden apple", Score: 0.3825
 - ID: food_1, Text: "fresh red apples", Score: 0.4809
 - ID: food_14, Text: "red fruit", Score: 0.5965
